In [2]:
import os
import pickle
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
import numpy as np
import pandas as pd
from torchvision.transforms import functional as TF
from PIL import Image
from libraries.bcosconv2d import NormedConv2d
import pydicom 
import random
import matplotlib.pyplot as plt

from collections import OrderedDict

from libraries.bcosconv2d import NormedConv2d
from libraries.bcoslinear import BcosLinear
from pooling.flc_bcosconv2d import ModifiedFLCBcosConv2d, ModifiedFLCASAPBcosConv2d

from libraries import augmentations



np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

# Paths
csv_path = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\grouped_data.csv"

#csv_path = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_labels.csv"
image_folder = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_images"
splits_path = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\splits\splits_balanced_fix.pkl"
model_path_flc = r"C:\Users\Admin\Documents\MasterThesis\results\Pneumonia\ResNet50_FLC\no_nosamp\seed_0\pneumonia_detection_model_resnet_bestf1_1.pth"
model_path_normal = r"C:\Users\Admin\Documents\MasterThesis\results\Pneumonia\ResNet50_BCos\no_nosamp\seed_0\pneumonia_detection_model_resnet_bestf1_1.pth"

with open(splits_path, 'rb') as f:
    splits = pickle.load(f)

class PneumoniaDataset(Dataset):
    def __init__(self, dataframe, image_folder, transform=None):
        self.data = dataframe
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_path = os.path.join(self.image_folder, f"{row['patientId']}.dcm")
        label = row['Target']
        patient_id = row['patientId']

        # Load DICOM file and process it into RGB format
        dicom = pydicom.dcmread(image_path)
        image = dicom.pixel_array
        image = Image.fromarray(image).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long), patient_id


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_worse = torch.hub.load('B-cos/B-cos-v2', 'resnet50', pretrained=True)

model_worse.layer2[0].conv2 = ModifiedFLCASAPBcosConv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2)
model_worse.layer2[0].downsample[0] = ModifiedFLCASAPBcosConv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), b=2)

model_worse.layer3[0].conv2 = ModifiedFLCASAPBcosConv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2)
model_worse.layer3[0].downsample[0] = ModifiedFLCASAPBcosConv2d(512, 1024, kernel_size=(1, 1), stride=(2, 2), b=2)

model_worse.layer4[0].conv2 = ModifiedFLCASAPBcosConv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2)
model_worse.layer4[0].downsample[0] = ModifiedFLCASAPBcosConv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), b=2)    

'''
model_worse.layer2[0].conv2 = ModifiedFLCBcosConv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2, transpose=True)
model_worse.layer2[0].downsample[0] = ModifiedFLCBcosConv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), b=2, transpose=False)

model_worse.layer3[0].conv2 = ModifiedFLCBcosConv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2, transpose=True)
model_worse.layer3[0].downsample[0] = ModifiedFLCBcosConv2d(512, 1024, kernel_size=(1, 1), stride=(2, 2), b=2, transpose=False)

model_worse.layer4[0].conv2 = ModifiedFLCBcosConv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2, transpose=True)
model_worse.layer4[0].downsample[0] = ModifiedFLCBcosConv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), b=2, transpose=False)    
'''
model_worse.fc.linear = NormedConv2d(2048, 2, kernel_size=(1, 1), stride=(1, 1), bias=False) # code from B-cos paper reused to adjust network
state_dict_worse = torch.load(model_path_flc, map_location=device)

model_worse.load_state_dict(state_dict_worse)
model_worse = model_worse.to(device)
model_worse.eval()

# Load model
model_better = torch.hub.load('B-cos/B-cos-v2', 'resnet50', pretrained=True)
model_better.fc.linear = NormedConv2d(2048, 2, kernel_size=(1, 1), stride=(1, 1), bias=False) # code from B-cos paper reused to adjust network
state_dict_better = torch.load(model_path_normal, map_location=device)

model_better.load_state_dict(state_dict_better)
model_better = model_better.to(device)
model_better.eval()


# Transformations
transform = transforms.Compose([
    transforms.ToTensor()  # Normalize with ImageNet stats
    ])

data = pd.read_csv(csv_path)
first_split = splits[0]
val_idx = first_split[1]  # Only use the validation indices from the 5th split
val_data = data.iloc[val_idx]
val_dataset = PneumoniaDataset(val_data, image_folder, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


directory = r"C:\Users\Admin\Documents\MasterThesis\comparison_images\FLC_Diff_Test"
os.makedirs(directory, exist_ok=True)

def visualize_explanation_difference(difference_tensor, save_path="difference_plot.png"):
    """Visualize and save the difference between explanation maps."""
    # Convert tensor to numpy array
    if difference_tensor.is_cuda:
        difference_array = difference_tensor.cpu().detach().numpy()
    else:
        difference_array = difference_tensor.numpy()

    # Remove singleton dimensions if needed
    difference_array = difference_array.squeeze()

    # Create visualization
    plt.figure(figsize=(10, 8))
    img = plt.imshow(difference_array, cmap='coolwarm')
    plt.colorbar(label='Difference Magnitude')
    plt.title("Explanation Difference (Actual - FLC)")
    plt.axis('off')
    
    # Save and show
    plt.savefig(save_path, bbox_inches='tight')
    plt.show()
    print(f"Difference plot saved to {save_path}")



i = 0
with torch.no_grad():
    for images, labels, patient_ids in val_loader:
        images, labels = images.to(device), labels.to(device)
        six_channel_images = []
        if i > 5:
            break
        for img_tensor in images:
            numpy_image = (img_tensor.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
            pil_image = Image.fromarray(numpy_image)
            transformed_image = model_worse.transform(pil_image)
            six_channel_images.append(transformed_image)
        six_channel_images = torch.stack(six_channel_images).to(device)
        outputs = model_worse(six_channel_images)  # Logits
        probs = torch.softmax(outputs, dim=1)  # Probabilities
        preds = torch.argmax(probs, dim=1)  # Binary predictions
        for image, patient_id in zip(six_channel_images, patient_ids):
            i += 1
            if i > 50:
              break
            image = image[None]
            expl_worse = model_worse.explain(image)
            filename = f"{patient_id}_worse_explanation.png"
            image_path_worse = os.path.join(directory, filename)

            plt.figure()
            plt.imshow(expl_worse["explanation"])
            plt.axis('off')
            plt.savefig(image_path_worse, bbox_inches="tight", pad_inches=0)
            plt.close()
        
        
            expl_better = model_better.explain(image)
            filename = f"{patient_id}_better_explanation.png"
            image_path = os.path.join(directory, filename)
                    
            plt.figure()
            plt.imshow(expl_better["explanation"])
            plt.axis('off')
            plt.savefig(image_path, bbox_inches="tight", pad_inches=0)
            plt.close()

            subtracted_expl = expl_better["contribution_map"] - expl_worse["contribution_map"]
            print(subtracted_expl)
            
            visualize_explanation_difference(subtracted_expl)
          


Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main


TypeError: correct_ASAP_padding_large.__init__() got an unexpected keyword argument 'transpose'

In [2]:
import torch
from libraries.bcosconv2d import NormedConv2d
model = torch.hub.load('B-cos/B-cos-v2', 'convnext_base', pretrained=True)      
#model.classifier[1] = NormedConv2d(2048, 2, kernel_size=(1, 1), stride=(1, 1), bias=False)
print(model)




Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main


BcosConvNeXt(
  (features): Sequential(
    (0): BcosConv2d(
      B=2,
      (linear): NormedConv2d(6, 128, kernel_size=(4, 4), stride=(4, 4), bias=False)
    )
    (1): DetachablePositionNorm2dNoBias((128,), eps=1e-06, elementwise_affine=True)
    (2): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): BcosConv2d(
            B=2,
            (linear): NormedConv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128, bias=False)
          )
          (1): DetachablePositionNorm2dNoBias((128,), eps=1e-06, elementwise_affine=True)
          (2): BcosConv2d(
            B=2,
            (linear): NormedConv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
          (3): BcosConv2d(
            B=2,
            (linear): NormedConv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequen

In [13]:
from torchvision.models import convnext_base
import torch.nn as nn

import torch
from torchvision.models import convnext_base, ConvNeXt_Base_Weights

# 1. Load pretrained ConvNeXt with LayerNorm
model = convnext_base(weights=ConvNeXt_Base_Weights.IMAGENET1K_V1)
#model.classifier[2] = torch.nn.Linear(1024, 2, bias=True)
print(model)


ConvNeXt(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (1): Permute()
          (2): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=128, out_features=512, bias=True)
          (4): GELU(approximate='none')
          (5): Linear(in_features=512, out_features=128, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (1): Permute()
          (2): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (3): Linear(